### **Your Name Here**

# Prelab 4 - Intro to Fitting Data


### Prelab 4 Contents

1. Noise, Data, Model
2. Residuals  
  2.1 Defining and Plotting Residuals  
  2.2 Summing Residuals  
  2.3 Comparing Residuals 
3. Model Fitting  
  3.1 Optimization  
  3.2 Overfitting  
  3.3 Fitting with error bars  
  3.4 Weighted Least-Squares fitting  
4. Fitting and Hubble's Law  

In [ ]:
#the usual import statements
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### 1. Noise, Data, Model

One of the most useful tools in an astronomer's toolkit is model fitting. At it's most basic level, this just means comparing data to a function and using any differences to gauge how fully that function describes the data. 

Let's start with simple linear fitting. To simulate some "real" data, we'll draw numbers from a model like y=mx+b, but then add some random noise to them. Luckily, numpy has lots of great random number generation functions that we'll use extensively later in the class. As you can see from the output, the code below will generate an array of 21 numbers randomly distributed between -0.5 and 0.5.

In [ ]:
#generate 21 random numbers from a flat distribution (all values equally probable) with values between 0 and 1. 
#subtract 0.5 so that the numbers now range from -0.5 and 0.5, which we'll call "noise"
noise = (np.random.rand(21)-0.5)
#eventually we'll add this to our model to synthesize data, but for now let's just print it so we can verify
noise

Now let's create a simple model. First we'll generate an independent variable, x, that ranges from 0 to 20

In [ ]:
x = np.arange(21)
x

Now let's generate a bunch of y points equal to twice the x values. In other words, following the function y=2x. This function is what we'll refer to as the "model" throughout, so we'll name it as such.

In [ ]:
model=2*x
model

Of course, even when physical laws are a simple as y=2x, in nature we never measure things so perfectly. Imperfections in our detectors, human error, unaccounted for physical effects, etc. all create "noise" such that real data NEVER perfectly matches a physical model. So we need some imperfect data to compare this model to if we're going to explore mdoel fitting. Let's make some synthetic "data" by adding the "noise" we've already created to our "model". 

In [ ]:
data=model+noise

And now let's plot it

In [ ]:
#plot synthetic data with points and model with dashed red line
plt.plot(x,data, 'bo', label='data')
plt.plot(x,model,'r--', label='model')
plt.xlim(0,20)
plt.xlabel("independent variable")
plt.ylabel("dependent variable")
plt.title("tight scatter")
plt.legend()

OK so that's a bit ridiculous. If you squint you can see that the values don't fall perfectly on the line, but clearly they're not very far off either. Let's make our "noise" more dramatic, and then plot again. 

In [ ]:
#make noisier noise (between -5 and 5)
noise2 = (np.random.rand(21)-0.5)*10
noise2

In [ ]:
#make and plot this new set of data
data2 = model+noise2
plt.plot(x,data2, 'bo', label='data')
plt.plot(x,model, 'r--', label='model')
plt.xlabel("independent variable")
plt.ylabel("dependent variable")
plt.title("larger scatter")
plt.legend()

---
Exercise 1 

---

Do the same thing yourselves for a slightly more complex "model" - one with a slope and an intercept. Generate an independent variable and a model that follows some y=mx+b functional form. Then, generate synthetic "data" with a reasonable scatter to mimic what you think "real" data might look like, and then plot the data and model on the same plot.

In [ ]:
#define independent variable

In [ ]:
#define model

In [ ]:
#define noise

In [ ]:
#create synthetic data

In [ ]:
#plot

---

## 2. Residuals 

### 2.1 Defining and Plotting Residuals

As discussed in the readings, when we have data and a model fit, one way to assess the quality of the fit is to look at the residuals, so let's do that. If your model is defined at all the same points as your data, this is simple. To make residuals, all you have to do is subract your model from your data. 

Note that in many cases this may not be the case, and you might have to "interpolate" your model to predict the value it would have at all the same x locations as your data points. We won't do this right now, but keep it in mind for the future.

In [ ]:
#define residuals
residuals = data2-model

#plot them
plt.plot(x, residuals, 'rx')
#plot a "perfect fit" line at residual value = 0
plt.plot([0,20],[0,0],'--')
plt.ylim(-5,5)
plt.xlim(0,20)
plt.ylabel('residual (data - model)')
plt.xlabel('independent variable')

---
### Exercise 2
---

What sorts of things should you look for in residual plots? What patterns indicate a high-quality fit, and what patterns might indicate that your model is not a good fit to the data? In your own words, describe at least two patterns for each case (good and bad fit).

***Your answer here***

---

### 2.2 Summing Residuals

Residuals are a great way to judge quality of fit qualitatively, and residual plote appear often in the astronomical literature, but when comparing multiple possible models to data we often want a ***quantitative** way to judge the quality of fit. 

The simplest of these metrics is just to sum the residuals. In this case, we want the DISTANCE from the line rather than the difference, because otherwise points above and below the line will cancel one another and we are looking for a measure of how far away from zero they are on average.

In [ ]:
#squaring and square rooting gives us only positive distances 
residuals_mag = np.sqrt((residuals)**2)
residuals_mag

In [ ]:
#then add them all up to get a total measure of the distances between the data points and the model
total_error_mag = sum(residuals_mag)
total_error_mag

More common in quality of fit metrics is to take the square of the distances between data points and the model, rather than just the distance. This gives a little extra penalty to points that are far away from the model.

In [ ]:
#or we can take the squares, as is more commonly done
residuals_sq = residuals**2
residuals_sq

In [ ]:
#then add them all up to get a total measure of the magnitudes
total_error_sq = sum(residuals_sq)
total_error_sq

These sums are the numbers that you're trying to minimize when doing "least-squares" fitting, which is the most common type of model fitting in astronomy. 

One important note is that the sum of the residuals or the squared residuals generates a number that has relatively little meaning in isolation. Indeed, the trick with quality of fit metrics is often the context. The number only means something relative to another fit of the same type applied to the same data. 

### 2.3. Comparing Residuals

The examples I've given so far have been pretty contrived. Data are of course not usually generated from a model. Rather, they are generated by the physical laws that govern the universe and we have to come up with the models that we think best describe them. Now, let's assume that I have only the data and no knowledge of the underlying model relationship

In [ ]:
#some data I collected
plt.plot(x, data2, 'go')
plt.xlabel("independent variable")
plt.ylabel("dependent variable")
plt.title("no fit")

In exploring the data and developing a model, I might first want to know something about the value of the correlation coefficient R for these two variables

In [ ]:
#there are lots of ways to do this in python. here's one
from scipy.stats.stats import pearsonr
#the output is the correlation coefficient R and the "p value", a measure of significance that we'll talk about later
pearsonr(x,data2)

I might have an idea of roughly what functional form the underlying physical law should take, or two different models that I want to test against one another. For example:

In [ ]:
#this sum of squares metric might also allow me to judge the quality of one model relative to another. For example:
model2 = 2.1*x-1
plt.plot(x,data2, 'go')
plt.plot(x,model)
plt.plot(x,model2,'r--')
plt.xlabel("independent variable")
plt.ylabel("dependent variable")
plt.title("potential fits")

They both look like reasonable matches to the data, so how do I know which one matches better?. I could start with the residuals:

In [ ]:
model2 = 2.1*x-1
plt.plot(x,data2-model, 'go', label='model 1')
plt.plot(x,data2-model2, 'rx', label='model 2')
plt.plot([0,20],[0,0],'k')
plt.xlabel("independent variable")
plt.ylabel("residual")
plt.legend
plt.title("potential fits")

If these models were more different from one another we might be able to judge which is better by eye, but in this case it's rather ambiguous from looking at the residuals which model is a better fit. Instead, we can use the sum of squared residuals as a quantitative metric. Since we're using the same metric to compare two models to the same data, these two numbers do now have meaning relative to one another. The one with the lower value is the better fit. 

In [ ]:
error1 = sum((model-data2)**2)
error2 = sum((model2-data2)**2)
print("sum of squares for model 1 (true) is ", error1)
print("sum of squares for model 2 is ",error2)

Note that if you execute all these cells multiple times, not infrequently the quality of fit metric for the alternate model will be better than the "true" model. The "truth" in the case of model fitting is elusive. We only know what the data tell us, and real data are never a perfect representation of truth. We can only do our best given the data we have and design other independent experiments to test our models. 

---
### Exercise 3
---

(a) Write a simple function with three required inputs - an array of independent variable values, the coefficients (slope, intercept) for a linear model, and an array of data values. The function should create a residual plot, and compute and print the sum of squared residuals. 

(b) Using your independent variable and data arrays from exercise 1 and your function from part (a), vary the slope of the model by hand until you acheive the lowest possible value of the sum of squared residuals. Keep the intercept fixed at the value you defined in exercise 1 and vary the slope until you have the "best fit" slope for your "data" to two decimal places. 

(c) Write a paragraph explaining why your "true" model (from which you generated the data, the functional form that you defined in Exercise 1) is not the same as the "best fit" model. Your explanation should integrate descriptions of the residual plots and the quantitative values of the sum of squared residuals for both models. What deeper truths does this reveal about the process of model fitting?

---

## 3. Model Fitting

### 3.1 Optimization

Of course there are more sophisticated ways to choose a model besides simple trial and error. At their most basic level, these are often built around the idea of minimizing the distances of the residuals from zero.

Python has lots of built-in functionalities for this kind of thing. My preferred methodm and the most generic/tunable choice, is using the scipy optimization module 

In [ ]:
#now let's try a more general model fitting function
import scipy.optimize as optimization

Under the optimization module, you have to define a general functional form for the fit line BUT NOT THE SPECIFIC VALUES FOR THE COEFFICIENTS, etc. 

For example, for linear (straight line) fits this could take two forms. A line without an intercept (a line starting at the origin)

$$y=mx$$

or a line with an intercept

$$y=mx+b$$

The function that we need to define and pass into scipy.optimize is just a python definition of these functions - something that takes in an independent variable (x) and the tunable parameters of the line (slope or slope and intercept) and returns the y value. 

In [ ]:

#line without an intercept (intercept zero)
def slopefunc(x,sl):
    return sl*x

#line with an intercept
def slopeintfunc(x,sl,incpt):
    return sl*x+incpt

The nice thing about scipy.optimize as opposed to certain built-in linear regression (straight line fitting) functions, is that this extends easily to more complicated functional forms and is not limited to straight line fits. For example, you can just as easily write a quadratic function of the form:

$$y=a+bx+cx^2$$

In [ ]:
#quadratic function
def quadfunc(x,a,b,c):
    return a+b*x+c*x*x

Now that we've defined the some functional forms for our fit lines, we can do the optimization itself with the function curve_fit, where the arguments are the name of the function we wrote describing the functional form of the line (in this case, a line with an intercept ```slopeintfunc```), and the independent (```x```) and dependent (```data2```) data arrays.

In [ ]:
#then use curve_fit
fit = optimization.curve_fit(slopeintfunc,x,data2)

The object returned by the ```curve_fit``` function (which we have here called ```fit```) is a tuple. The first element contains the "best fit" values for whatever the tunable parameters are in your model (in our case slope and intercept). 

In [ ]:
#best fit parameters (sl, int)
fit[0]

As you can see, the first element of the tuple is itself a list containing the best fit values for the tunable parameters. Let's extract them individually.

In [ ]:
best_slope = fit[0][0]
best_intercept = fit[0][1]

The next element of the tuple contains what's called the covariance matrix, which can also be quite useful. We will explore covariance matrices in more detail in Unit 2 of the course. 

In [ ]:
#covariance matrix
fit[1]

The best sanity check after you've done a fit is to overplot it on the data. Does it match the data well?

In [ ]:
#plot the data
plt.plot(x,data2, 'go', label='data')
#plot the model
plt.plot(x, slopeintfunc(x,best_slope,best_intercept), label='linear model (m=1.96, b=0.21)')
#labels
plt.legend()
plt.xlabel("independent variable")
plt.ylabel("dependent variable")
plt.title("least squares fit")

---
### Exercise 4
---

(a) Define for yourself a general cubic function of the form 
$$y=a+bx+cx^2+dx^4$$ 
(b) using curve_fit, extract the optimal values for the a, b, c, and d coefficients.   
(c) overplot this fit (define the coefficients in your legend label) on our made up data (x, data2).   
(d) describe in words how this fit differs from the linear fit I did for you above. Which is a better fit to the data in your opinion and what criteria are you using to judge?  


In [ ]:
#your cubic function definition goes here

In [ ]:
#your curve fit goes here

In [ ]:
#your plot goes here

***Your explanation for part d goes here***

---
### 3.2 Overfitting

Since we can define functions to arbitrary dimensions, fitting can definitely can get a bit out of control. For example:

In [ ]:
def tenparamfunc(x,a,b,c,d,e,f,g,h,i,j):
    return a+b*x+c*x**2+d*x**3+e*x**4+f*x**5+g*x**6+h*x**7+i*x**8+j*x**9

In [ ]:
fit2 = optimization.curve_fit(tenparamfunc,x,data2)
fit2[0]

In [ ]:
plt.plot(x,data2, 'go')
c = fit2[0]
plt.plot(x, tenparamfunc(x,c[0],c[1],c[2],c[3],c[4],c[5],c[6],c[7],c[8],c[9]))
plt.xlabel("independent variable")
plt.ylabel("dependent variable")
plt.title("fit for function with ten parameters")

A simple but important rule of thumb is:

> ***The number of parameters in your model should be <<< the number of data points***

### 3.3 Fitting with error bars

Often we know enough about how our measurements are taken that we can assign "error bars" or "uncertainties" to our measurements. These uncertainties can take two forms. 

***Homoschedastic*** errors are the same for every point. These tend to happen when you have some standard uncertainty associated with a certain type of measurement or measurement apparatus. 

***Heteroschedastic*** errors vary from one data point to the next and are generally a property of the data themselves. In the example below, the error on each value is equal to its square root. This is basically the defintion of uncertainty for poisson-based statistical processes, and is therefore very common in astronomy. 

The cell below defines a homoschedastic and a heteroschedastic array of errors, one for each point in our dataset.

In [ ]:
# equal errors (homoschedastic)
#an array of threes
errors_uniform = np.ones(21)*3

#errors that vary (heteroschedastic)
errors_poisson = np.sqrt(data2)

To plot data points with error bars, there is a very handy built in function called errorbar that allows you to specfy values for ```yerr``` and ```xerr``` (y and x error bars). These should be arrays or lists with the same number of entries as you have datapoints.

In [ ]:
#visualize the homoschedastic errors
plt.errorbar(x,data2,yerr=errors_uniform, fmt='go')
plt.xlim(0,20)
plt.xlabel("independent variable")
plt.ylabel("dependent variable")
plt.title("homoschedastic error bars")

In [ ]:
plt.errorbar(x,data2,yerr=errors_poisson, fmt='go')
plt.xlim(-1,21)
plt.xlabel("independent variable")
plt.ylabel("dependent variable")
plt.title("heteroschedastic error bars")

### 3.4 Weighted Least Squares Fitting

If we want to take the uncertainty in each of our data points into consideration in calculating goodness of fit, we can extend this to assigning "weights" to each data point. 

Since larger error bars indicate greater uncertainty, these data points should be assigned less weight than other data points with smaller error bars. 

A weight is just like a coefficient in front of the (data-model)$^2$ calculation typical to least squares. More formally:

$$ Q = \sum_{i=1}^nw_i[y_i-f(x_i,\beta)]^2$$

Where $x_i$ is the independent variable, $y_i$ are the observed values, $f(x_i,\beta)$ is the model with some set of parameters $\beta$ and $w_i$ are the weights for each datapoint

A common weight is the reciprocal of the error value squared, or $\frac{1}{\sigma^2}$. Sigma here is the value of the error bar and is not to be confused with a standard deviation, though standard deviation values are often assigned as errors. 

Let's do this for our example of heteroschedastic error bars above

In [ ]:
lsq_weighted=sum(1/errors_poisson**2*(data2-model)**2)
lsq_weighted

Oops what happened? Well, the model value at x=0 is 0 in this case, and the errors are too, so our 1/errors_poissson statement becomes problematic because we can't divide by zero. 

We can fix this by removing the datapoint from consideration (indeed it's rare that we measure something to be zero anyway, so it was a bit contrived to begin with). 

Let's do that and also compute a new dependent ```model3```, simulated dataset ```data3``` and poisson error array ```errors_poisson3```

In [ ]:
#new independent array, starting at 1
x3=np.arange(20)+1
#new y-2x dependent variable for that array
model3=2*x3
#random noise
noise3 = (np.random.rand(20)-0.5)*10
#simulated data
data3= 2*x3+noise3
#poisson errors 
errors_poisson3 = np.sqrt(data3)

Now let's compute the ***weighted*** sum of squares vlaue, which we could use as a quality of fit metric to compare two models where we know something about uncertainties. 

In [ ]:
#compute the weighted least-squares value 
lsq_weighted=sum(1/errors_poisson3**2*(data3-model3)**2)
lsq_weighted

Similarly, we can build in the uncertainties/weights when we do the least squares fit to the data. This functionality is built into the curve_fit function through the optional ```sigma``` parameter). 

Let's compute the fit again here with and without the weights, so that we can compare the two fits visually. 

In [ ]:
fit_weighted = optimization.curve_fit(slopeintfunc,x3,data3, sigma=errors_poisson3)
fit_unweighted = optimization.curve_fit(slopeintfunc,x3,data3)

In [ ]:
#plot data
plt.errorbar(x3,data3,yerr=errors_poisson3, fmt='go')
plt.xlim(0,21)
plt.ylim(0,50)
#plot weighted and unweighted fits
plt.plot(x3, slopeintfunc(x3,fit_weighted[0][0],fit_weighted[0][1]), label='weighted')
plt.plot(x3, slopeintfunc(x3,fit_unweighted[0][0],fit_unweighted[0][1]), 'r--', label='unweighted')
#labels
plt.legend(loc='lower right',)
plt.xlabel("independent variable")
plt.ylabel("dependent variable")
plt.title("weighted vs. unweighted fits")

In [ ]:
optimization.curve_fit?

## 4. Fitting and Hubble's law

For lab 4, you will work with a classmate to process and fit both the original data leading to the derivation of an important astrophysical law known as "Hubble's Law", which basically amounts to a discovery that the universe is expanfing, and more modern measurements of the same quantities. The next two exercises will get the tedious bits of reading in and familiarizing yourself with the data so that you can do the hard parts (manipulating and fitting) done in lab. 

--- 
## Exercise 5
---

In the cell below, I have transcribed the data from Edwin Hubble's original 1928 paper "A relation between distance and radial velocity among extra-galactic nebulae", available [here](https://www.pnas.org/content/pnas/15/3/168.full.pdf).

a.  Open the original paper (in the Drive folder for this prelab). Use it and your knowledge of Python code to decipher what each line in the next two code cells is doing. Add a comment at the top of each line stating what it is doing and/or where in the paper it came from.   
b. Create a scatter plot from Hubble's data. To make a scatterplot in python, you use the same plt.plot function that we used for line graphs last week except after the x and y arguments, you add a string describing the type of plotting symbol that you want. [Here](https://matplotlib.org/3.1.1/api/markers_api.html) is a list of plot symbols. Note that you can combine these with colors so, for example, 'go' is green circles and 'rx' is red xs. Give your plot a title and axis labels to match Hubble's original.  
c. Write code that will print each entry in the list obj_list on its own line (you will need this for exercise 2, below).

In [ ]:
NGC_nos = [6822,598,221,224,5457,4736,5194,4449,4214,
        3031,3627,4826,5236,1068,5055,7331,4258,
        4151,4382,4472,4486,4649]
obj_list = ['SMC', 'LMC']
for i in np.arange(len(NGC_nos)):
    obj_list.append('NGC '+str(NGC_nos[i]))

In [ ]:
dists = np.array([0.032,0.034,0.214,0.263,0.275,0.275,0.45,0.5,0.5,0.63,0.8,0.9,0.9,
         0.9,0.9,1.0,1.1,1.1,1.4,1.7,2.0,2.0,2.0,2.0])#Mpc
vels = np.array([170.,290,-130,-70,-185,-220,200,290,270,200,300,-30,650,150,500,920,450,500,500,960,500,850,800,1000]) #km/sec

In [ ]:
#plot goes here

In [ ]:
#loop to print names goes here

---
    
## Exercise 6
---
Now, let's pull modern data for Hubble's galaxies. Copy and paste the list from Exercise 5c into the query form [here](http://ned.ipac.caltech.edu/forms/gmd.html). ***Before you click "Submit Query"***, scroll to the check boxes at the bottom of the page and make sure to check ***only*** the following:
  *  User Input Object Name
  *  Redshift
  *  Redshift Uncertainty  
  And in the bottom right panel:  
  *  Metric Distance
  *  Mean
  *  Standard Deviation
  *  Number of measurements

Open a simple text editor application (e.g. "TextEdit" on a Mac) and copy and paste the table into it. Save it as cat.txt and place it in your google drive. You may want to be sure that you're saving it as plain rather than formatted text as well. Most text editors have an option to do this in the format menu or at the file saving stage.

The code cells below will "read in" the data using a python package called Pandas that we will learn about in great detail in the coming weeks. For now, just execute the cell below, which will create python lists stored in variables with descriptive names from your cat.txt file. 

a)Describe in words at least two patterns that you note in the tabular data  
b) Make a histogram for each of the following quantities: redshift, redshift_uncert, dist, and dist_uncert. All your plots should have axis labels, and for the histograms you should play around with the number of bins until you can justify your choice for this value.  Discuss and compare the shapes of the distributions for each of the quantities in general, qualitative terms.   
c) Plot the uncertainty in redshift as a function of redshift for these galaxies and the uncertainty in distance as a function of distance. What patterns do you notice, if any in the relationships between these quantities and their uncertainties?  

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#modify the path below to match wherever you've stored your cat.txt file (this is just for your main Google Drive folder)
path_to_cat = '/content/drive/MyDrive/cat.txt'

In [ ]:
#import the necessary module
import pandas
#define columns
cols = ['Obj Name', 'Redshift', 'Redshift Uncert', 'Dist Mean (Mpc)', 'Dist Std Dev (Mpc)', 'Num Obs']
#read in the file
df = pandas.read_csv(path_to_cat, delimiter ='|', skiprows=3, header = 0, names = cols, skipinitialspace=True)

#extract the relevant values as lists
redshift = df["Redshift"].tolist()
redshift_uncert = df["Redshift Uncert"].tolist()
dists2 = df["Dist Mean (Mpc)"].tolist()
dists2_uncert = df["Dist Std Dev (Mpc)"].tolist()

In [ ]:
#display table (python "data frame" object)
df

***Answer to Part a***

In [ ]:
#plots for part b - redshift

In [ ]:
#plots for part b - redshift uncertainty

In [ ]:
#plots for part b - distance

In [ ]:
#plots for part b - distance uncertainty

***Part B explanation***

In [ ]:
#part c scatter plot 1

In [ ]:
#part c scatter plot 2

***Part C explanation***

# Sumbitting Prelabs and Labs for Grading

Before submitting any Google Colab notebook for grading, please follow the following steps

**1) Try running everything in one go (Runtime menu -> Restart and run all)**

Make sure the entire notebook runs from start to finish. If necessary, comment out any un-executable cells from the instructions portion of the lab so the whole notebook will execute in one go. 

**2) Restart the kernel (Runtime menu --> Restart Runtime).**

**3) Clear all output (Edit --> clear all outputs).**

**4) Make sure the names of all group members are in a markdown cell at the top of the file and submit the notebook through the Moodle link for this Lab**